In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from scipy.stats import linregress
import seaborn as sns
# Model 3: Miniaturized 'real' CRISPEY Experiment

#bottleneck_counts and extract_DNA are the same, copied for readability
def bottleneck_counts(counts, bottleneck_size=800000):
    #samples strains based from a poisson with lambda = their fraction in the library * bottleneck_size
    #modeling the culture bottleneck when cells are diluted
    lambdas = counts/counts.sum()*bottleneck_size
    return np.array([np.random.poisson(i) for i in lambdas])

def extract_DNA(counts, extraction_barcodes):
    
    '''
    samples strains based on a poisson distribution with lambda = their fraction in the library * extraction_barcodes
    modeling the culture bottleneck
    
    Parameters
    ----------
    counts : Numpy array (int)
        Counts for strains at the end of a time point 
    extraction_barcodes : int
        number of total barcodes to be extracted (average coverage in extraction * )
   
    Returns
    -------
    Numpy array
       Array containing the final sequenced counts for each strain

    '''
    lambdas = counts/counts.sum()*extraction_barcodes
    return np.array([np.random.poisson(i) for i in lambdas])

def PCR_and_sequence(counts, num_reads, pcr_cycles, efficiency):
    '''
    simulates PCR process with some error (no clue what would be reasonable there)
    then samples from the pcr counts as a poisson for each strain with
    lambda = their fraction in the pcr counts * sequencing depth
    
    Parameters
    ----------
    counts : Numpy array (int)
        Counts for strains after DNA extraction 
    num_reads : int
        Rough number of sequencing reads
    pcr_cycles: int
        Number of PCR cycles
    
    Returns
    -------
    Numpy array
       Array containing the final sequenced counts for each strain

    '''
    pcr = np.copy(counts)
    for i in range(pcr_cycles):
        num_amplified = np.random.binomial(pcr,efficiency)
        pcr = (pcr-num_amplified)+num_amplified*2
    sequence_lambdas = pcr/pcr.sum()*num_reads
    return np.array([np.random.poisson(i) for i in sequence_lambdas])

def calculate_fitness(s, e, num_gens):
    #calculates fitness as a function of start and end counts
    #currently not used
    try:
        fitness = 2**(np.log2((e-s)/s +1)/num_gens)
        if fitness <.001:
            fitness = None
    except:
        fitness = None
    return fitness

def simulate_crispey(fitness_size, bottleneck, \
                     num_strains = 400, num_tp = 6, gens_per_tp = 5, \
                     extraction_coverage = 4, sequencing_reads = 1000000, pcr_cycles = 24,
                    num_reps = 1, pcr_efficiency = .9):
    '''

    Assuming a lot of poisson and other processes, simulates a Crispey competition

    Parameters
    ----------
    fitness_size : float
        Scale for exponential distribution of relative fitness differences from 1 for strains in the competition. 
    bottleneck : int
        Rough number of yeast passaged into the next timepoint, also used to derive the initial counts for strains
    num_strains: int
        Number of unique strains with fitness values being measured
    num_tp: int
        Number of time points in the competition when yeast are passaged/sampled.
    gens_per_tp: int
        Number of doublings a neutral yeast strain would go through in each timepoint
    extraction_coverage: float
        Average coverage of each strain in the extracted DNA (number of yeast genomes for each edit)
    sequencing_reads: int
        Rough number of reads to be sequenced
    pcr_cycles: int
        Number of PCR cycles performed on extracted DNA
    num_reps: int
        Number of replicate competitions performed
    Returns
    -------
    output_df: Pandas DataFrame
        DataFrame containing the sequenced counts of each strain in each time point for each replicate,
        along with the true fitness value for each strain

    See Also
    --------
    bottleneck_counts: helper function for the culturing bottleneck step
    extract_DNA: helper function to perform DNA extraction step
    PCR_and_sequence: helper function to perform PCR and sequencing steps
    '''
    #Having total starting counts equal to bottleneck size--realistic
    starting_count_size = bottleneck/num_strains

    #Sampling starting counts from exponential distribution to very uneven library distribution
    starting_counts = np.round(np.random.exponential(starting_count_size, num_strains))
    
    #Sampling fitness effects from exponential distribution
    fit_effects = np.random.exponential(fitness_size,num_strains)
    
    #Making fitness effects not all positive
    selection_coefs = np.array([each*random.choice([-1,1]) for each in fit_effects])
    fitness = 1+selection_coefs
    output_df = pd.DataFrame({'True_fitness':fitness, 'T0_counts':starting_counts})
    
    #having all replicates start with the exact same starting counts
    #moderately unrealistic
    prev_counts = [starting_counts]*num_reps
    for i in range(1, num_tp):
        for j in range(num_reps):
            #Simulating drift as a function of bottleneck size
            drift = np.random.normal(0, 1/np.sqrt(bottleneck), num_strains)
            
            #simulating the growth of the strains in a timepoint
            true_end_counts = prev_counts[j]*(2*fitness)**gens_per_tp+drift
            true_end_counts = np.round(true_end_counts)
            
            extracted_counts = extract_DNA(true_end_counts, num_strains*extraction_coverage)
            sequence_counts = PCR_and_sequence(extracted_counts, sequencing_reads, pcr_cycles, pcr_efficiency)
            
            output_df['T'+str(i)+'_rep'+str(j)+'_counts'] = sequence_counts
            
            prev_counts[j] = bottleneck_counts(true_end_counts,bottleneck)
    output_df = output_df.fillna(0)
    return output_df



def significant_fitness_regression(crispey_df, replicate):
    '''
    Modifies a simulated Crispey dataset in place, adding regression slopes and p values for each strain
    
    Parameters
    ----------
    crispey_df : Pandas DataFrame
        Simulated Crispey Counts Dataset 
    replicate : int
        Replicate to calculate regressions on
   
    Returns
    -------
    Nothing
    
    '''
    #modifies in place
    c2 = crispey_df.filter(regex = 'rep'+str(replicate))
    for column in c2.filter(regex = 'counts'):
        c2[column] = np.log2((c2[column]+0.5)/c2[column].sum())
    for i, row in c2.iterrows():
        model_df = pd.DataFrame()
        model_df['generations'] = range(0,25,5)
        model_df['logfreq'] = row.filter(regex = 'counts').values.astype(float)

        res = linregress(model_df['generations'],model_df['logfreq'])
        crispey_df.loc[i,'regression_fitness_rep'+str(replicate)] = 1+res.slope
        crispey_df.loc[i,'regression_pvalue_rep'+str(replicate)] = res.pvalue
        
def generate_fishers_table(hit_df, truefit,est_fit): 
    '''Generates a 2x2 contingency table for true fitness and fitness measured by simulation
    Parameters
    ----------
    hit_df : Pandas DataFrame
        Simulated Crispey hits dataframe with true and estimated fitness 
    truefit : str
        column name for true fitness effects
    est_fit : str
        column name for estimated fitness effects
   
    Returns
    -------
    pandas DataFrame containing a 2x2 contingency table for the direction of fitness effects'''
    
    pospos = len(hit_df[(hit_df[truefit]>1)&(hit_df[est_fit]>1)])
    posneg = len(hit_df[(hit_df[truefit]>1)&(hit_df[est_fit]<1)])
    negpos = len(hit_df[(hit_df[truefit]<1)&(hit_df[est_fit]>1)])
    negneg = len(hit_df[(hit_df[truefit]<1)&(hit_df[est_fit]<1)])
    return pd.DataFrame({'index':['Negative Estimated Fitness', 'Positive Estimated Fitness'],'Negative True Fitness':[negneg, negpos], 'Positive True Fitness':[posneg,pospos]})

def calculate_power(simdf, minimum_fitness, p = 0.05, truefit = 'True_fitness',\
                    est_fit= 'regression_fitness_rep0', est_p = 'regression_pvalue_rep0'):
    '''Calculates power for a given replicate simulation
    with regression to detect fitness effects a certain distance from 1
    for a given p value threshold. (Also requires effect to be in the correct direction)
    
    Parameters
    ----------
    simdf : Pandas DataFrame
        Simulated Crispey dataframe with true and estimated fitness 
    minimum_fitness: float
        minimum value that true fitness effects must be away from 1 (neutral) to be considered
        a hit for power analysis
    p: float
        p-value for an estimated fitness effect to be considered a hit
    truefit : str
        column name for true fitness effects
    est_fit : str
        column name for estimated fitness effects
    est_p : str
        column name for estimated fitness p values
    Returns
    -------
    float : power as a decimal (0-1.0)
    '''
    #getting all "real" hits in the simulation
    all_true_hits = simdf[abs(1-simdf[truefit])>minimum_fitness]
    #getting all "real" hits in the simulation with a p value less than p 
    #from the fitness estimation and where the estimated fitness agrees with
    #the direction of the true fitness
    detected_true_hits = simdf[(abs(1-simdf[truefit])>minimum_fitness)&\
                               (simdf[est_p]<p)&(simdf[truefit]*simdf[est_fit]>0)]
    power = len(detected_true_hits)/len(all_true_hits)
    return power
    
def true_fitness_comp(simdf, rep, name_of_sim = ''):
    '''plotting function for plotting scatter plot of measured fitness versus true fitness for
    a CRISPEY simulation
    
    '''
    fig, ax = plt.subplots()

    scatter = ax.scatter(simdf['True_fitness'],simdf['regression_fitness_rep'+str(rep)],c = simdf['sigreg'+str(rep)])


    plt.axvline(1, c = 'red', linestyle = 'dotted')
    plt.axhline(1, c = 'red', linestyle = 'dotted')
    plt.xlabel('True Fitness')
    plt.ylabel('Estimated Fitness')
    legend1 = ax.legend(*scatter.legend_elements(),
                        loc="lower right", title="p<.05")
    ax.add_artist(legend1)
    plt.title(name_of_sim+' Rep '+str(rep+1)+' Simulation Regression Analysis'  )
    
def est_fitness_comp(simdf,  reps = [0,1], name_of_sim = '', filter_sig = []):
    
    '''plotting function for plotting scatter plot of measured fitness in different replicates
    a CRISPEY simulation'''
    #filter_sig can be rep number, None, or 'both'
    if len(filter_sig) == 2:
        for rep in filter_sig:
            plotdf = simdf[(simdf['sigreg'+str(reps[0])])&(simdf['sigreg'+str(reps[1])])]
    elif len(filter_sig):
        plotdf = simdf[(simdf['sigreg'+str(filter_sig[0])])]
    else:
        plotdf = simdf.copy()
    
    fig, ax = plt.subplots()

    scatter = ax.scatter(plotdf['regression_fitness_rep'+str(reps[0])],plotdf['regression_fitness_rep'+str(reps[1])])


    plt.axvline(1, c = 'red', linestyle = 'dotted')
    plt.axhline(1, c = 'red', linestyle = 'dotted')
    plt.xlabel('Rep1 Fitness')
    plt.ylabel('Rep2 Fitness')
    plt.title(name_of_sim+' Rep1 vs Rep2 Estimated Fitness')

In [ ]:
#Doing a parameter sweep for the extraction coverage (how many yeast have dna prepped),
#and sequencing depth, and storing lots of diagnostics for each simulation
#slow, next step would be optimizing the regression step or plugging into limma-voom or deseq

sig_level = .05
d = {'extraction coverage':[], 'total reads':[], 'replicate_correlation':[], \
     'rep1_true_fitness_corr':[],'rep2_true_fitness_corr':[],'replicate_corr_both_hits':[],\
    'rep1_true_fitness_corr_both_hits':[],'rep2_true_fitness_corr_both_hits':[],'number_of_hits':[],
     'power_rep1':[], 'power_rep2':[]}
for extraction_level in [100, 500,1000,2000,4000,5000, 10000,25000,100000]:
    for sequencing_depth in [100,250,500,1000,2000,3000,5000,10000]:
        for i in range(100):
            sim = simulate_crispey(.005,800000, num_strains = 400, num_reps =2, \
                               extraction_coverage = extraction_level, sequencing_reads=sequencing_depth*400)
            significant_fitness_regression(sim,0)
            significant_fitness_regression(sim,1)
            sim['sigreg0'] = sim['regression_pvalue_rep0']<sig_level
            sim['sigreg1'] = sim['regression_pvalue_rep1']<sig_level
            
            both_hits = sim[(sim['sigreg0'])&(sim['sigreg1'])]
            bcorr = both_hits.filter(regex = 'fitness').corr()
            sim_fit_corr = sim.filter(regex = 'fitness').corr()
            d['number_of_hits'].append(len(both_hits))
            d['replicate_corr_both_hits'].append(bcorr.loc['regression_fitness_rep0','regression_fitness_rep1']**2)
            d['rep1_true_fitness_corr_both_hits'].append(bcorr.loc['regression_fitness_rep0','True_fitness']**2)
            d['rep2_true_fitness_corr_both_hits'].append(bcorr.loc['regression_fitness_rep1','True_fitness']**2)
            d['extraction coverage'].append(extraction_level)
            d['total reads'].append(sequencing_depth*400)
            d['replicate_correlation'].append(sim_fit_corr.loc['regression_fitness_rep0','regression_fitness_rep1']**2)
            d['rep1_true_fitness_corr'].append(sim_fit_corr.loc['regression_fitness_rep0','True_fitness']**2)
            d['rep2_true_fitness_corr'].append(sim_fit_corr.loc['regression_fitness_rep1','True_fitness']**2)
            d['power_rep1'].append(calculate_power(sim, .01, sig_level))
            d['power_rep2'].append(calculate_power(sim, .01, sig_level,est_fit= 'regression_fitness_rep1', est_p = 'regression_pvalue_rep1'))
        print('done with extraction level '+ str(extraction_level)+' and sequencing level '+str(sequencing_depth))
   

In [ ]:


sim_results = pd.DataFrame(d)

In [ ]:
s2 = sim_results.groupby(['extraction coverage', 'total reads']).agg('mean')

In [ ]:
sim_results.index = sim_results['extraction coverage']

In [ ]:
sim_results.drop('extraction coverage', inplace = True, axis = 1)

In [ ]:
sns.heatmap(s2.unstack()['rep2_true_fitness_corr'])
plt.title('R^2 b/w True fitness and one Rep Measured Fitness for all simulated strains 100 sims each')

In [ ]:
sns.heatmap(s2.unstack()['rep1_true_fitness_corr_both_hits'], annot = s2.unstack()['number_of_hits'])

In [ ]:
sns.heatmap(s2.unstack()['replicate_correlation'])
plt.title('R^2 b/w replicates for all simulated strains 100 sims each')

In [ ]:
sns.heatmap(s2.unstack()['replicate_corr_both_hits'])